# Cortes
>$n$: Quantidade de tipos de itens  
>$m$: Quantidade de padrões de corte  

## Variáveis
>$x_j$: Quantidade que será utilizada do padrão de corte $j$, para $j = 1,\ldots, m$  
>$s_j = 
  \begin{cases}
    1 : \text{Caso o padrão } j \text{ seja utilizado} \\
    0 : \text{Caso não seja}
  \end{cases}
$

## Parâmetros
>$D_i$: Demanda do item $i$, para $i = 1,\ldots, n$  
>$P_{ij}$: Padrão de corte $i$ do tipo de item $j$, para $i = 1,\ldots, n$ e $j = 1,\ldots, m$  
>$R$: Máximo permitido de padrões de corte  

## Modelo  

\begin{align*}
    \hbox{min} \ \ 
        & \sum_{j=1}^m x_i \\
        
    \hbox{s.a.} \ \ 
        & \sum_{j=1}^m P_{ij} x_j \geq D_i && i = 1, \ldots, n \\
        & \sum_{j=1}^m s_j \leq R                              \\
        & x_j \leq M s_j                   && j = 1, \ldots, m \\
        & M = \sum_{i=1}^n D_i                                 \\
        & s_j \in \{0, 1\}                 && j = 1, \ldots, m \\
        & x_{j} \in +\mathbb{Z}            && j = 1, \ldots, m \\
\end{align*}

In [10]:
from itertools import product


def cut_patterns(M: int, C: list[int]) -> list[list[int]]:
    corte_minimo = M - min(C)
    cortes = product(*(range(M//i + 1) for i in C))
    x = [i for i in cortes if corte_minimo < sum(a*b for a, b in zip(i, C)) <= M]

    return x

In [11]:
import pandas as pd
from IPython.display import display

def create_table(M: int, C: list[int]):
    patterns = cut_patterns(M, C)
    df = pd.DataFrame(patterns, columns=C)
    df["Resto"] = M - df.mul(C).sum(axis=1)
    # display(df)
    
    return df.to_string()

In [12]:
from pymprog import *


def modelo_cortes(M, R, C, D):
    P = [*zip(*cut_patterns(M, C))]

    n = len(D)
    m = len(P[0])

    begin("cortes")

    x = var("x", m, int)
    s = var("s", m, bool)

    minimize( sum(x) )

    for i in range(n):
        sum(P[i][j] * x[j] for j in range(m)) >= D[i]

    M = sum(D)
    for j in range(m):
        x[j] <= s[j] * M

    sum(s) <= R

    solver(int, tm_lim= 120_000)
    solve()

    obj = vobj()

    end()

    return obj, x, P

In [13]:
import os
import pandas as pd

resumo = pd.DataFrame(index=[
    "Tamanho da matriz",
    "Tamanho dos itens menores",
    "Limite de Padrões de corte",
    "Qtn. de padrões de corte",
    "Total de papel usado",
    "Qtn. de papel desperdiçada"
])

for file in os.listdir("./input/"):
    with open(f"input/{file}", "r") as f:
        M = int(f.readline())

        R = int(f.readline())

        C, D = [], []
        for i in f.readlines():
            c, d = map(int, i.split())
            C.append(c)
            D.append(d)

        print(f"\n{file:=^75}")

        # create_table(M, C)
        obj, varx, P = modelo_cortes(M, R, C, D)
        P = [*zip(*P)]

        with open(f"output_b/{file}", "w") as f:
            f.write(create_table(M, C) + "\n\n")

            f.write(f"Tamanho da bobina matriz: {M}\n")
            f.write(f"Tamanho das bobinas menores: {C}\n")
            f.write(f"Demandas: {D}\n\n")

            f.write(f"Quantidade de bobinas usadas: {int(obj)}\n\n")

            desperdicio_total = 0

            for i, x in enumerate(varx):
                if x.primal:
                    resto = M - sum(a*b for a, b in zip(P[i], C))
                    desperdicio_total += int(x.primal) * resto

                    f.write(f"Padrão {i:>2}: {P[i]} resto {resto}\n")
                    f.write(f"Quantidade usada: {int(x.primal)}\n")
                    f.write(f"Quantidade desperdiçada: {int(x.primal) * resto}\n")
                    f.write(f"Quantidade satisfeita de cada demanda: {tuple(j * int(x.primal) for j in P[i])}\n\n")

            f.write(f"Quantidade total de papel gasto: {int(obj) * M}\n")
            f.write(f"Quantidade desperdiçada: {desperdicio_total}\n")

            resumo[file] = [M, C, R, len(cut_patterns(M, C)), int(obj) * M, desperdicio_total]


==============================inst_200_5.txt===============================
GLPK Simplex Optimizer 5.0
57 rows, 102 columns, 282 non-zeros
      0: obj =   0.000000000e+00 inf =   1.209e+04 (5)
     13: obj =   3.446229167e+03 inf =   0.000e+00 (0)
*    20: obj =   3.338166667e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
57 rows, 102 columns, 282 non-zeros
102 integer variables, 51 of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+    20: mip =     not found yet >=              -inf        (1; 0)
Solution found by heuristic: 3582
Solution found by heuristic: 3478
+  1724: >>>>>   3.340000000e+03 >=   3.339000000e+03 < 0.1% (1346; 17)
+ 36752: mip =   3.340000000e+03 >=   3.339000000e+03 < 0.1% (1629; 28349)
+ 73879: mip =   3.340000000e+03 >=   3.339000000e+03 < 0.1% (5644; 44044)
+ 89724: mip =   3.340000000e+03 >=   3.339000000e+03 < 0.1% (7020; 50494)
+ 99692: >>>>>   3.339000000e+03 >=   3.339000000e+03   0.0%

In [14]:
display(resumo.T)

,Tamanho da matriz,Tamanho dos itens menores,Limite de Padrões de corte,Qtn. de padrões de corte,Total de papel usado,Qtn. de papel desperdiçada
inst_200_5.txt,200,"[21, 24, 43, 79, 98]",5,51,667800,18950
inst_130_6.txt,130,"[19, 22, 28, 54, 63, 64]",7,48,667550,5956
inst_200_4.txt,200,"[21, 43, 79, 98]",4,14,676200,13826
inst_110_5.txt,110,"[10, 12, 17, 18, 21]",5,229,118580,0
inst_100_4.txt,100,"[15, 16, 31, 40]",5,24,219400,10943
inst_250_5.txt,250,"[33, 43, 52, 60, 87]",5,64,771000,11108
inst_100_6.txt,100,"[15, 16, 18, 31, 32, 40]",6,91,313000,6116
inst_130_4.txt,130,"[19, 28, 54, 63]",5,14,400530,6938
inst_250_6.txt,250,"[33, 43, 52, 53, 60, 87]",7,119,772500,2998
inst_200_6.txt,200,"[21, 24, 43, 53, 79, 98]",6,87,816800,15757
